In [ ]:
# Standard Library Imports
import time
import colorsys
from itertools import combinations

# Scientific Computing
import numpy as np
import sympy as sp
from scipy.spatial import ConvexHull

# Plotting Libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import plotly.graph_objects as go

# PyCalphad (Thermodynamics Calculations & Plotting)
from pycalphad import Database, calculate, equilibrium, variables as v
from pycalphad.plot.utils import phase_legend
from pycalphad import ternplot

# Computational Geometry
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union

# Color Processing
from skimage.color import deltaE_ciede2000, rgb2lab

In [ ]:
# Load database and choose the phases that will be plotted
db = Database(r'../TDDatabaseFiles_temp/alfe.tdb')

phases = list(db.phases.keys())
constituents = list(db.elements)
legend_handles, color_dict = phase_legend(phases)

print(phases)
print(constituents)

In [ ]:
def fit_poly_x2y2_sympy(x_data, y_data, z_data):
    """
    Fit a polynomial of the form:
      f(x, y) = a00 + a10*x + a01*y + a20*x^2 + a11*x*y + a02*y^2 +
                a21*x^2*y + a12*x*y^2 + a22*x^2*y^2
    to the given data using least squares, and return a Sympy expression.
    
    Parameters:
        data (np.ndarray): A (n x 3) array where each row is [x, y, z].
        
    Returns:
        expr (sympy.Expr): A Sympy expression representing the fitted polynomial.
    """
    
    # Construct the design matrix
    A = np.column_stack([
        np.ones_like(x_data),    
        x_data,                  
        y_data,                  
        x_data**2,               
        y_data**2,               
    ])
    
    # Solve the least squares problem to get the coefficients
    coeffs, residuals, rank, s = np.linalg.lstsq(A, z_data, rcond=None)
    
    # Create sympy symbols for x and y
    x, y = sp.symbols('x y')
    
    # Define the list of polynomial terms in the same order as in A:
    terms = [
        1,          
        x,          
        y,          
        x**2,       
        y**2,       
    ]
    
    # Build the polynomial expression by summing coeff * term for each term.
    expr = sum(sp.Float(coeff) * term for coeff, term in zip(coeffs, terms))
    
    return sp.simplify(expr), residuals

In [ ]:
def lower_convex_hull(points):
    '''
    Calculate the lower convex hull, assuming the last dimension represents energy.

    Parameters:
        points (array): Points in N-dimensional space, with the last dimension representing energy.

    Returns:
        lower_hull (array): Array of indices describing the points that form the lower convex hull.
    '''
    processing_points = points.copy()

    # Check if the projected points are collinear
    projected_points = processing_points[:, :-1]
    transformed_points = projected_points - projected_points[0]
    if np.linalg.matrix_rank(transformed_points) == 1:
        idx = np.argsort(np.linalg.norm(transformed_points, axis=1))
        bp = np.array([idx[0], idx[-1]])
        processing_points = processing_points[:, 1:]

    else:
        bp = ConvexHull(points).simplices.flatten()
    
    fake_points = processing_points[bp].copy()
    fake_points[:, -1] += 500000  # offset to create "upper" points
    processing_points = np.vstack((processing_points, fake_points))

    hull = ConvexHull(processing_points)
    simplices = hull.simplices

    mask = np.all(simplices < len(points), axis=1)
    lower_hull = simplices[mask]

    return lower_hull

In [ ]:
# Calculate all the enthalpy as a funciton of the entropy and composition
def format_enthalpy(entropy_result, enthalpy_result):
    X = entropy_result.X.sel(component='FE').values[0, 0, :, :].flatten()
    S = entropy_result.SM.values[0, 0, :, :].flatten()
    H = enthalpy_result.HM.values[0, 0, :, :].flatten()

    sort_idx = np.argsort(X)
    X_sorted = X[sort_idx]
    H_sorted = H[sort_idx]
    S_sorted = S[sort_idx]

    sort_idx = np.argsort(S)
    X_sorted = X_sorted[sort_idx]
    S_sorted = S_sorted[sort_idx]
    H_sorted = H_sorted[sort_idx]

    return X_sorted, S_sorted, H_sorted

enthalpy_phase_dict = dict()
for phase_name in phases:
    # Only computing 10 teperature points because the plotting struggles
    temp_points_count = 40
    entropy_result = calculate(db, constituents, phase_name, P=101325, T=np.linspace(300, 2000, temp_points_count), output = "SM")
    enthalpy_result = calculate(db, constituents, phase_name, P=101325, T=np.linspace(300, 2000, temp_points_count), output = "HM")

    X, S, H = format_enthalpy(entropy_result, enthalpy_result)
    enthalpy_phase_dict[phase_name] = (X, S, H)

In [ ]:
# Keep only the equilibrium enthalpy points for fitting
eq_enthalpy_phase_dict = dict()
for phase in phases:
    print(phase)
    X, Y, Z = enthalpy_phase_dict[phase]

    # Get the points into the lower hull
    points = np.column_stack((X, Y, Z))
    simplices = lower_convex_hull(points)

    # Keep only the points that are in the lower hull
    points = points[np.unique(simplices.ravel())]
    eq_enthalpy_phase_dict[phase] = (points[:, 0], points[:, 1], points[:, 2])

In [ ]:
# These are the 'good' fitting phases
phase_poly_dict = dict()
for phase_name in phases:
    # These are the phases that don't have a good fit
    if phase_name in ['AL2FE', 'AL13FE4', 'AL5FE2']:
        continue
    
    X, Y, Z = eq_enthalpy_phase_dict[phase_name]

    # Compute the fitted polynomial
    energy_polynomial, res = fit_poly_x2y2_sympy(X, Y, Z)
    print(phase_name)
    display(energy_polynomial)
    print("residual:", res)
    phase_poly_dict[phase_name] = energy_polynomial

In [ ]:
import re

def sympy_to_desmos(expr):
    """
    Convert a sympy expression into a format suitable for Desmos.
    
    This function:
      - Replaces '**' followed by an integer (including negative integers)
        with '^' and wraps the integer in curly brackets.
      - Replaces any remaining '**' with '^'.
      - Removes '*' used as the multiplication operator.
      
    Parameters:
        expr: A sympy expression or any object that can be converted to a string.
        
    Returns:
        A string formatted for use in Desmos.
    """
    # Convert the expression to a string
    s = str(expr)
    
    # Replace exponentiation of an integer (or negative integer) with a curly-braced exponent
    s = re.sub(r'\*\*(-?\d+)', r'^{\1}', s)
    
    # Replace any remaining exponentiation operators (if any)
    s = s.replace('**', '^')
    
    # Remove the multiplication operator
    s = s.replace('*', '')
    
    return s


In [ ]:
fig = go.Figure()
for phase_name in ['LIQUID','B2_BCC']:
    
    X, Y, Z = eq_enthalpy_phase_dict[phase_name]
    selected_color = color_dict[phase_name]

    # fig.add_trace(go.Scatter3d(
    #         x=X, y=Y, z=Z,
    #         mode='markers',
    #         name=phase_name,
    #         marker=dict(color=selected_color, size=1)
    #     ))

    # Compute the fitted polynomial
    energy_polynomial = phase_poly_dict[phase_name]
    # Rounding the polynomial
    energy_polynomial = energy_polynomial.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))
    print(sympy_to_desmos(energy_polynomial))

    x_mesh, y_mesh = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 100, 100))
    z_mesh = sp.lambdify((sp.symbols('x'), sp.symbols('y')), energy_polynomial, 'numpy')(x_mesh, y_mesh)

    fig.add_trace(go.Surface(
        x=x_mesh,
        y=y_mesh,
        z=z_mesh,
        showscale=False,
        colorscale=[[0, selected_color], [1, selected_color]]
    ))

fig.update_layout(
    scene=dict(
        xaxis_title="X(FE)",
        yaxis_title="Entropy (J/mol)",
        zaxis_title="Enthalpy (J/mol)"
    ),
    title="Equilibrium Enthalpy Surfaces"
)

fig.show()

In [ ]:
from sympy import symbols, diff

def projection_function(f1, f2):
    '''
    Computes the projection of f2 onto f1.
    
    Parameters:
        f1 : sympy expression

        f2 : sympy expression
    
    Returns:
        proj : sympy expression

        variables : tuple

        pvariables : tuple
    '''
    free_symbols_f1 = f1.free_symbols
    free_symbols_f2 = f2.free_symbols

    variables = tuple(free_symbols_f1.intersection(free_symbols_f2))
    pvariables = tuple(symbols(f'{symbol}_p') for symbol in variables)

    sum_terms = [(variable - pvariables[i])*diff(f1, variable).subs(variable, pvariables[i]) for i, variable in enumerate(variables)]
    proj = f2 - f1.subs(dict(zip(variables, pvariables))) - sum(sum_terms)

    return proj.expand(), variables, pvariables

In [ ]:
def recursive_discriminant(expr, vars):
    '''
    Takes the discriminant over all variables in a given expression.

    Parameters:
        expr (sympy expression): The expression to take the discriminant of.

    Returns:
        discriminant (sympy expression): The discriminant of the expression.
    '''
    vars_list = list(vars)
    def recursive_discriminant_helper(expr, vars):
        # if there are no variables, return the expression
        if vars == []:
            return expr
        
        for var in vars_list:
            if expr.has(var):
                vars_list.remove(var)
                return sp.discriminant(recursive_discriminant(expr, vars_list), var)
            
    return recursive_discriminant_helper(expr, vars_list)

In [ ]:
poly1 = phase_poly_dict['LIQUID'].replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))
poly2 = phase_poly_dict['B2_BCC'].replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))
display(poly1)

In [ ]:
disc_funcs = []
for pair in [(poly1, poly2), (poly2, poly1)]:
    proj, vars, pvars = projection_function(pair[0], pair[1])
    disc = recursive_discriminant(proj, vars)
    print(sympy_to_desmos(disc))
    disc_func = sp.lambdify(pvars, disc, 'numpy')
    disc_funcs.append(disc_func)

In [ ]:
# Define the range for x and y and create a mesh
x_mesh, y_mesh = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 100, 100))

for func in disc_funcs:
    Z = func(x_mesh, y_mesh)
    plt.contour(x_mesh, y_mesh, Z, levels=[0])
plt.grid()
plt.show()

In [ ]:
fig = go.Figure()
x_mesh, y_mesh = np.meshgrid(np.linspace(0, 1, 400), np.linspace(0, 100, 400))
for phase_name in ['LIQUID','B2_BCC']:
    
    X, Y, Z = eq_enthalpy_phase_dict[phase_name]
    selected_color = color_dict[phase_name]

    # Compute the fitted polynomial
    energy_polynomial = phase_poly_dict[phase_name]
    # Rounding the polynomial
    energy_polynomial = energy_polynomial.replace(lambda term: term.is_Number, lambda term: int(round(term, 0)))
    energy_func = sp.lambdify((sp.symbols('x'), sp.symbols('y')), energy_polynomial, 'numpy')

    z_mesh = energy_func(x_mesh, y_mesh)

    fig.add_trace(go.Surface(
        x=x_mesh,
        y=y_mesh,
        z=z_mesh,
        showscale=False,
        colorscale=[[0, selected_color], [1, selected_color]]
    ))

for i, disc_func in enumerate(disc_funcs):
    # Create contour plot
    z_contour = disc_func(x_mesh, y_mesh)
    contour_set = plt.contour(x_mesh, y_mesh, z_contour, levels=0)
    plt.close()

    all_segments = []
    for seg_level in contour_set.allsegs:
        # seg_level is a list of arrays; each array is shape (N, 2)
        for seg in seg_level:
            all_segments.append(seg)

    data = all_segments[1]
    
    z_vals = energy_func(data[:, 0], data[:, 1]) - 20000
    fig.add_trace(
        go.Scatter3d(
            x=data[:, 0],
            y=data[:, 1],
            z = z_vals,
            mode="lines"
        )
    )
        
fig.update_layout(
    scene=dict(
        xaxis_title="X(FE)",
        yaxis_title="Entropy (J/mol)",
        zaxis_title="Enthalpy (J/mol)",
        # zaxis=dict(range=[-20000, 60000]),
    ),
    title="Equilibrium Enthalpy Surfaces"
)

fig.show()